In [26]:
###########################################################################################################
# 
#                      Interfaces Gráficas para Instrumentación y Control
#
###########################################################################################################

In [27]:
import tkinter as tk
from tkinter import *
from tkdial import Dial # pip install tkdial

root = tk.Tk()
root.title('Interfaz para vehículos')
root.iconbitmap('C:/Users/ASUS/Pictures/symbol_light_S.ico') #
root.geometry("420x200")


dial_1 = Dial(root, color_gradient=("green", "cyan"),
             text_color="black", text="Current: ", unit_length=10, radius=50)
dial_1.place(x=10,y=10)

dial_2 = Dial(root, color_gradient=("green", "cyan"),
             text_color="black", text="Voltage: ", unit_length=10, radius=50)
dial_2.place(x=150,y=10)

dial_3 = Dial(root, color_gradient=("green", "cyan"),
             text_color="black", text="Frequency: ", unit_length=10, radius=50)
dial_3.place(x=290,y=10)

root.mainloop()

In [32]:
import tkinter
from tkdial import Jogwheel

app = tkinter.Tk()
app.title('Interfaz para vehículos')
app.iconbitmap('C:/Users/ASUS/Pictures/symbol_light_S.ico')
app.geometry("250x300")

knob = Jogwheel(app)
knob.grid()

app.mainloop()

In [33]:
import customtkinter #pip install customtkinter
from tkdial import Jogwheel

app = customtkinter.CTk()
app.title('Interfaz para vehículos')
app.iconbitmap('C:/Users/ASUS/Pictures/symbol_light_S.ico')
app.geometry("150x400")

wheel_1 = Jogwheel(app, radius=200, fg="#045252", scale_color="white",
                text=None, button_radius=10)
wheel_1.set_mark(0,100, "green")

wheel_1.grid()

wheel_2 = Jogwheel(app, radius=200, fg="#045252", scale_color="white", start_angle=0,
                   end_angle=360, start=0, end=200, text="Volume: ", button_radius=10)
wheel_2.set_mark(0,50, "blue")
wheel_2.set_mark(50, 90, "green")
wheel_2.set_mark(90, 150, "orange")
wheel_2.set_mark(150, 200, "red")
wheel_2.grid()

app.mainloop()

In [34]:
import tkinter
from tkdial import Meter

root = tkinter.Tk()
root.title('Interfaz para vehículos')
root.iconbitmap('C:/Users/ASUS/Pictures/symbol_light_S.ico')
root.geometry("300x300")

dial = Meter(root)
dial.pack(padx=10, pady=10)

root.mainloop()

In [ ]:
import customtkinter
from tkdial import Meter

app = customtkinter.CTk()
app.title('Interfaz para vehículos')
app.iconbitmap('C:/Users/ASUS/Pictures/symbol_light_S.ico')
app.geometry("800x350")

meter1 = Meter(app, radius=300, start=0, end=160, border_width=0,
               fg="black", text_color="white", start_angle=270, end_angle=-270,
               text_font="DS-Digital 30", scale_color="white", needle_color="red")
meter1.set_mark(140, 160) # set red marking from 140 to 160
meter1.grid(row=0, column=1, padx=20, pady=30)

meter2 = Meter(app, radius=260, start=0, end=200, border_width=5,
               fg="black", text_color="white", start_angle=270, end_angle=-360,
               text_font="DS-Digital 30", scale_color="black", axis_color="white",
               needle_color="white")
meter2.set_mark(1, 100, "#92d050")
meter2.set_mark(105, 150, "yellow")
meter2.set_mark(155, 196, "red")
meter2.set(80) # set value
meter2.grid(row=0, column=0, padx=20, pady=30)

meter3 = Meter(app, fg="#242424", radius=300, start=0, end=50,
               major_divisions=10, border_width=0, text_color="white",
               start_angle=0, end_angle=-360, scale_color="white", axis_color="cyan",
               needle_color="white",  scroll_steps=0.2)
meter3.set(15)
meter3.grid(row=0, column=2, pady=30)

app.mainloop()

In [ ]:
#############################################################################################################
#
#                                 Interacción con Hardware
#
#############################################################################################################

In [ ]:
######################################## Packages Loading #######################################################

import serial # require pip install PySerial for Serial communication
from tkinter import *
import tkinter as tk 
import time   #for delay functions

################################# GUI Windown logo and name configuration ########################################

root = Tk()
root.iconbitmap('C:/Users/ASUS/Pictures/symbol_light_S.ico') #
root.wm_title('Name of the Company')
root.resizable(width=False, height=False)
root.geometry("380x130")

########################################  Frame Configuration ####################################################

mainFrame = Frame(root, bd=10, bg='gainsboro',relief=RIDGE)
mainFrame.grid()

innerFrame = Frame(mainFrame, bd=5, bg='gainsboro', relief=RIDGE)
innerFrame.grid()

########################################  Serial port configuration ##############################################

ser = serial.Serial('COM5', 9600)  # open serial port
time.sleep(2)                      # wait 2 seconds

############################################### Functions set #####################################################

def turn_LED_ON():
    ser.write(b'H')

def turn_LED_OFF(): 
    ser.write(b'L')

############################################### Buttons Widget#######################################################

btn_On = Button(innerFrame, text="LED ON", width=10, height =2, font=('arial',18,'bold'), bd=7, bg='gainsboro', command=turn_LED_ON)
btn_On.grid(row=0,column = 0)

btn_Off = Button(innerFrame, text="LED OFF", width=10, height =2, font=('arial',18,'bold'), bd=7, bg='gainsboro', command=turn_LED_OFF)
btn_Off.grid(row=0,column = 2)

root.mainloop()

In [ ]:
#############################################################################################################
#
#                                             Vision por Computador
#
#############################################################################################################

In [1]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk

WIDTH, HEIGHT = 800, 600

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Camera Viewer")
        self.root.geometry(f"{WIDTH}x{HEIGHT}")

        self.cap = None
        self.running = False

        self.label = tk.Label(root)
        self.label.pack(expand=True)

        controls = tk.Frame(root)
        controls.pack(pady=10)

        tk.Button(controls, text="Start", command=self.start_camera).pack(side="left", padx=10)
        tk.Button(controls, text="Stop", command=self.stop_camera).pack(side="left", padx=10)

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    def start_camera(self):
        if self.running:
            return  # already running

        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Could not open camera.")
            return

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

        self.running = True
        self.update_frame()

    def update_frame(self):
        if not self.running or self.cap is None:
            return

        ret, frame = self.cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)

            self.label.imgtk = imgtk
            self.label.configure(image=imgtk)

        self.root.after(15, self.update_frame)

    def stop_camera(self):
        self.running = False

        if self.cap:
            self.cap.release()
            self.cap = None

        self.label.configure(image="")
        self.label.imgtk = None

    def on_close(self):
        self.stop_camera()
        self.root.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = CameraApp(root)
    root.mainloop()